# Essential Medicines Optimization

The paper "**_Combining retrosynthesis and mixed-integer optimization for minimizing the chemical
inventory needed to realize a WHO essential medicine list_**" by Gao, Coley, Struble, Li, Qian, Green
and Jensen, published in
[**_Reaction Chemistry and Engineering_**](https://pubs.rsc.org/en/content/articlelanding/2020/re/c9re00348g) addresses the problem of ensuring access to critical medicines.

The idea is to stock some limited set of starting or seed chemicals, which are relatively simple,
relatively stable, and readily available. This supply of seed chemicals is then used to synthesize
the desired medicines on site. Their approach is to use AI models for retrosynthesis to identify 
possible synthesis trees (or pathways) starting from seed chemicals. Each tree is scored for
feasibility or likelihood of success. Each seed chemical is scored for suitability, which could be
based on stability, availability, complexity, cost, etc. The optimization task is to choose one tree
for each result medicine so that, in aggregate, the quality of the selected trees is high while the
number and complexity of the seed chemicals is low.

The site https://github.com/Coughy1991/Combined_synthesis_planning contains the particular trees
considered for the paper as a python pickle file. The data includes almost 200,000 trees with a
total of about 2.83 million chemical nodes and 1.77 million reaction nodes. The GitHub site also
contains a lengthy python script, in the form of a Jupyter notebook, that loads the pickle file,
translates the contents to a form that the CPLEX solver can accept and executes the optimization.
The translation to the CPLEX form takes almost an hour and is quite complex.

This notebooks starts from their synthesis trees to implementation the MIP optimization
portion of their technique using Rexl module functionality and the
[Gurobi](https://www.gurobi.com/) optimizer.

The formulation here is in a much simpler form and at a higher level than the python code.

## From RetroSynthesis Generated Reaction Trees

The trees are represented as two tables, `ChemNodes` and `ReacNodes`.

These tables were generated by retrosynthesis AI models as described above. We wrote
some python that loads the authors' original pickle files, and writes the data out to
two parquet files (one for chemicals and one for reactions). We then wrote a Rexl script
that loads these parquet files, transforms the information a bit, and writes them as
**R**exl **Bin**ary files (known as **rbin**).

This notebook starts from these two rbin files, one for the chemicals used and produced
and one for reactions that use and produce the chemicals.

In [1]:
play T1 as ReadRbin("../Data/ChemNodes.rbin");
play T2 as ReadRbin("../Data/ReacNodes.rbin");
finish T1;
finish T2;

ChemNodes := T1.Data;
ReacNodes := T2.Data;

Display some basic information: types, number of things, etc.

In [2]:
"Type for ChemNodes: " & ChemNodes->GetType();
"Type for ReacNodes: " & ReacNodes->GetType();
"Number of chemical nodes: " & ChemNodes->Count()->ToText();
"Number of reaction nodes: " & ReacNodes->Count()->ToText();
"Number of trees: " & ChemNodes.tid->Distinct()->Count()->ToText();

Type for ChemNodes: {ID:i8, rid:i8, rid_maker:i8, smiles:s, tid:i8}*
Type for ReacNodes: {ID:i8, cid:i8, score:r8, tid:i8}*
Number of chemical nodes: 2829838
Number of reaction nodes: 1777235
Number of trees: 193597


## Sample Tree

Showing the SMILES representation (Simplified Molecular Input Line Entry System).

In [3]:
// UDF to show a reaction tree (or forest).
func ShowTree(chems) :=
    Fold(chems, seq: [], seq ++ [{Maker:rid_maker, Nest:First(seq, rid = Maker).Nest + 1 ?? 1, V:smiles}])
    +>{ ID: # }
    ->SortDown(ID)
    ->ScanZ(as r, need: 0, With(bit: 1 shl Nest, need band (bit - 1) bor bit),
        r+>{
            Need: need,
            Pre: Range(1, Nest)->("   " if (need band (1 shl it) = 0) else "|  ")->Concat("") & "+- "
        })
    ->SortUp(ID)
    ->(Pre & V)
    ->Concat("\n");

In [4]:
ChemNodes
    ->TakeIf(tid = 163446)
    ->ShowTree();

+- CCOC(=O)C(CCc1ccccc1)NC(C)C(=O)N1CCCC1C(=O)O
   +- CCOC(=O)C(CCc1ccccc1)NC(C)C(=O)N1CCCC1C(=O)OCc1ccccc1
      +- CCOC(=O)C(CCc1ccccc1)NC(C)C(=O)Cl
      |  +- CCOC(=O)C(CCc1ccccc1)NC(C)C(=O)O
      |     +- CCOC(=O)C(CCc1ccccc1)NC(C)C(=O)OC(C)(C)C
      |        +- CCOC(=O)C(CC)NC(C)C(=O)OC(C)(C)C
      |        |  +- CC(C)(C)O
      |        |  +- CCOC(=O)C(CC)NC(C)C(=O)O
      |        +- Ic1ccccc1
      +- O=C(OCc1ccccc1)C1CCCN1
         +- CC(C)(C)OC(=O)N1CCCC1C(=O)OCc1ccccc1
            +- CC(C)(C)O
            +- O=C(OCc1ccccc1)C1CCCN1C(=O)O
               +- O=C(Cl)C1CCCN1C(=O)O
               +- OCc1ccccc1


# Declare the Optimization Plan

* Parameters
* Computed Constants
* Free Variables
* Derived Variables
    * Constraints
    * Measures (potential objectives)
    * Others (for display)

In [5]:
Retro := module {
    // *** Parameters ***

    // For mixing measures in the objective.
    param Lambda := 0.1;

    // Chemical nodes. This is a sample record of the correct type for defining the module.
    // Could also make the LHS @ChemNodes, referencing the global, but it is often better
    // to define the module with minimal data (of the correct type).
    param ChemNodes := [ { ID: 0, rid: -1, rid_maker: 0, tid: 0, smiles: "O=C1c2c(O)cccc2Cc2cccc(O)c21" } ];

    // Reaction nodes. This is a sample record of the correct type for defining the module.
    param ReacNodes := [ { ID: 0, cid: 0, tid: 0, score: 0.48797703872669806 } ];

    // *** Computed constants - most of them are compound (tables, etc). ***

    // Group the chemical and reaction nodes by tree id.
    const ChemNodesByTid := ChemNodes->GroupBy(tid, Chems);
    const ReacNodesByTid := ReacNodes->GroupBy(tid, Reacs);

    // Join to get tree representations.
    // Sort by tree id.
    // Append the result smiles and tree quality score (smaller better).
    const Trees :=
        KeyJoin(ChemNodesByTid as t, ReacNodesByTid, tid, tid, t+>{ Reacs })
            ->SortUp(tid)
            +>{ ResSmiles: Chems->First(rid < 0).smiles, Score: Reacs->Sum(-Ln(score max 0.05)), };
    const NumTrees := Trees->Count();

    // Group the trees by result chemical.
    const TreesByRes := Trees->GroupBy([key] ResSmiles, [auto] Trees);

    // Group the chemical nodes by their smiles. Append whether it is a seed chemical.
    const Chemicals :=
        ChemNodes->GroupBy(
            [key] smiles,
            [auto] Nodes,
            [group] IsSeed: Any(group, rid_maker < 0));

    // Get the seed chemicals, assign each an index and compute the complexity score.
    // Note: could use the "Synthetic Complexity Score" rather than this crude measure of complexity.
    const SeedChems :=
        Chemicals->TakeIf(IsSeed)
            ->SortUp(smiles)
            +>{ SeedIdx: #, ComplexityScore: 1 + Ln(1 + smiles.Len) };
    constant NumSeeds := SeedChems->Count();

    // *** Variables ***

    // An indicator (bool) variable for each tree. This corresponds to x_ij in the paper.
    var IncludeTree from Tensor.Fill(false, NumTrees);

    // An indicator (bool) variable for each seed chemical. This corresponds to y_k in the paper.
    var IncludeSeed from Tensor.Fill(false, NumSeeds);

    // *** Constraints ***

    // Need one tree for each result chemical.
    constraint OneTreePerRes := TreesByRes->ForEach(Trees->Sum(IncludeTree[tid]) = 1);

    // Must include needed seed chemicals. That is, include seed iff there are included trees that use it.
    constraint NeededSeeds :=
        SeedChems->ForEach(
            IncludeSeed[SeedIdx] <= Sum(Nodes, IncludeTree[tid]) <= Nodes->Count() * IncludeSeed[SeedIdx]);

    // *** Measures ***

    // The number of included seed chemicals (smaller better).
    measure SeedCount := SeedChems->Sum(IncludeSeed[SeedIdx]);

    // The aggregate complexity of the included seed chemicals (smaller better).
    measure SeedScore := SeedChems->Sum(ComplexityScore * IncludeSeed[SeedIdx]);

    // The aggregate quality score of the included trees (smaller better).
    measure QualityScore := Trees->Sum(Score * IncludeTree[tid]);

    // Mixed objective (smaller better).
    measure MixedScore := Lambda * SeedScore + (1 - Lambda) * QualityScore;
    
    // *** Derived Variables for Display ***

    let ShowSeedCount := "Seed Count (smaller better): " & SeedCount->ToText();
    let ShowSeedScore := "Seed Score (smaller better): " & SeedScore->ToText();
    let ShowQualScore := "Tree Quality Score (smaller better): " & QualityScore->ToText();
    let ShowScores := "\n" & ShowSeedCount & "\n" & ShowSeedScore & "\n" & ShowQualScore;
};

# Combine the AI-Generated Reaction Trees with Optimization Plan

In [6]:
R := Retro with { @ChemNodes, @ReacNodes };

In [7]:
"Number of Trees: " & R.NumTrees->ToText();
"Number of Chemicals: " & R.Chemicals->Count()->ToText();
"Number of Seed Chemicals: " & R.NumSeeds->ToText();
"Number of Result Chemicals: " & R.TreesByRes->Count()->ToText();

Number of Trees: 193597
Number of Chemicals: 8485
Number of Seed Chemicals: 3340
Number of Result Chemicals: 99


### Result Chemical SMILES and Number of Trees

In [8]:
R.TreesByRes
    ->{ ResSmiles, Num: Count(Trees) }
    ->Sort(Num)

Seq<{i8,str}>
   0) { Num: 9001, ResSmiles: CCN(CC)CCNC(=O)c1ccc(N)cc1 }
   1) { Num: 8395, ResSmiles: CC(C)NCC(O)COc1ccc(CC(N)=O)cc1 }
   2) { Num: 7991, ResSmiles: OCCN1CCN(CCCN2c3ccccc3Sc3ccc(C(F)(F)F)cc32)CC1 }
   3) { Num: 7890, ResSmiles: N=C(N)c1ccc(OCCCCCOc2ccc(C(=N)N)cc2)cc1 }
   4) { Num: 7884, ResSmiles: CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1 }
   5) { Num: 7805, ResSmiles: CCOC(=O)C1(c2ccccc2)CCN(C)CC1 }
   6) { Num: 7779, ResSmiles: CCCCN1CCCCC1C(=O)Nc1c(C)cccc1C }
   7) { Num: 7370, ResSmiles: CCOC(=O)C(CCc1ccccc1)NC(C)C(=O)N1CCCC1C(=O)O }
   8) { Num: 6958, ResSmiles: CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23 }
   9) { Num: 6600, ResSmiles: CN1C2CCC1CC(OC(=O)C(CO)c1ccccc1)C2 }
  10) { Num: 6594, ResSmiles: Cc1ccc(N(CC2=NCCN2)c2cccc(O)c2)cc1 }
  11) { Num: 6498, ResSmiles: CC1=C(C(=O)O)N2C(=O)C(NC(=O)C(N)c3ccccc3)C2SC1 }
  12) { Num: 5570, ResSmiles: O=C(CCCN1CCC(O)(c2ccc(Cl)cc2)CC1)c1ccc(F)cc1 }
  13) { Num: 5419, ResSmiles: CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12 }
  14) {

# Optimize Various Ways

In [9]:
Solver := "gurobi";

### Best tree quality score

In [10]:
#!time
Best_Qual := R->Minimize(QualityScore, Solver);
Best_Qual.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 235
Seed Score (smaller better): 894.4138304160446
Tree Quality Score (smaller better): 102.08880129959611


Wall time: 8697.3008ms

Try again for better timing, since the first run often takes extra time.

In [11]:
#!time
Best_Qual := R->Minimize(QualityScore, Solver);
Best_Qual.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 235
Seed Score (smaller better): 894.4138304160446
Tree Quality Score (smaller better): 102.08880129959611


Wall time: 8335.6604ms

### Best seed score

In [12]:
#!time
Best_SeedScore := R->Minimize(SeedScore, Solver);
Best_SeedScore.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 171
Seed Score (smaller better): 637.3073256362173
Tree Quality Score (smaller better): 337.25036592857305


Wall time: 42466.9824ms

### Fewest seed chemicals

In [13]:
#!time
Best_SeedCount := R->Minimize(SeedCount, Solver);
Best_SeedCount.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 171
Seed Score (smaller better): 655.0467856628211
Tree Quality Score (smaller better): 347.85245899326594


Wall time: 31543.4862ms

Note: "best seed score" produced the same number of seed chemicals with better seed score and similar tree quality. Clearly seed count is not the best measure to use.

### Various lambda values
Smaller lambda emphasizes tree quality, larger lambda emphasizes seed simplicity.

In [14]:
// Should be same as Best_Qual.
#!time
Best_00 := R=>{Lambda: 0.0}->Minimize(MixedScore, Solver);
Best_00.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 235
Seed Score (smaller better): 894.4138304160446
Tree Quality Score (smaller better): 102.08880129959611


Wall time: 11295.616ms

In [15]:
#!time
Best_10 := R=>{Lambda: 0.1}->Minimize(MixedScore, Solver);
Best_10.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 199
Seed Score (smaller better): 754.6998044562729
Tree Quality Score (smaller better): 104.002431612898


Wall time: 13091.5306ms

In [16]:
#!time
Best_20 := R=>{Lambda: 0.2}->Minimize(MixedScore, Solver);
Best_20.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 196
Seed Score (smaller better): 741.5745775317198
Tree Quality Score (smaller better): 106.09093667179195


Wall time: 15344.8912ms

In [17]:
#!time
Best_30 := R=>{Lambda: 0.3}->Minimize(MixedScore, Solver);
Best_30.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 191
Seed Score (smaller better): 724.9396129461886
Tree Quality Score (smaller better): 110.88031602715259


Wall time: 17728.0461ms

In [18]:
#!time
Best_40 := R=>{Lambda: 0.4}->Minimize(MixedScore, Solver);
Best_40.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 188
Seed Score (smaller better): 705.889808031162
Tree Quality Score (smaller better): 120.81888023300853


Wall time: 20750.916ms

In [19]:
#!time
Best_50 := R=>{Lambda: 0.5}->Minimize(MixedScore, Solver);
Best_50.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 178
Seed Score (smaller better): 672.00319895559
Tree Quality Score (smaller better): 148.89253564882432


Wall time: 25668.5095ms

In [20]:
// Should be same as Best_SeedScore.
#!time
Best_100 := R=>{Lambda: 1.0}->Minimize(MixedScore, Solver);
Best_100.ShowScores;

Solver: Gurobi

Seed Count (smaller better): 171
Seed Score (smaller better): 637.3073256362175
Tree Quality Score (smaller better): 330.1806962394589


Wall time: 48445.3426ms